实验10 LDA

1、基于LDA分析题目-词矩阵，并与SVD结果比较。

In [5]:
#官网文档：https://lda.readthedocs.io/en/latest/getting_started.html
import numpy as np
import lda
#需要将datasets更换为自定义的数据集
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
X.shape
X.sum()
model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
model.fit(X)  # model.fit_transform(X) is also available
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 395
INFO:lda:vocab_size: 4258
INFO:lda:n_words: 84010
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -1051748
INFO:lda:<10> log likelihood: -719800
INFO:lda:<20> log likelihood: -699115
INFO:lda:<30> log likelihood: -689370
INFO:lda:<40> log likelihood: -684918
INFO:lda:<50> log likelihood: -681322
INFO:lda:<60> log likelihood: -678979
INFO:lda:<70> log likelihood: -676598
INFO:lda:<80> log likelihood: -675383
INFO:lda:<90> log likelihood: -673316
INFO:lda:<100> log likelihood: -672761
INFO:lda:<110> log likelihood: -671320
INFO:lda:<120> log likelihood: -669744
INFO:lda:<130> log likelihood: -669292
INFO:lda:<140> log likelihood: -667940
INFO:lda:<150> log likelihood: -668038
INFO:lda:<160> log likelihood: -667429
INFO:lda:<170> log likelihood: -666475
INFO:lda:<180> log likelihood: -665562
INFO:lda:<190> log likelihood: -664920
INFO:lda:<200> log likelihood: -664979
INFO:lda:<210> log likelihood: -664722
INFO:lda:<220> log likelihood: -

Topic 0: british churchill sale million major letters west
Topic 1: church government political country state people party
Topic 2: elvis king fans presley life concert young
Topic 3: yeltsin russian russia president kremlin moscow michael
Topic 4: pope vatican paul john surgery hospital pontiff
Topic 5: family funeral police miami versace cunanan city
Topic 6: simpson former years court president wife south
Topic 7: order mother successor election nuns church nirmala
Topic 8: charles prince diana royal king queen parker
Topic 9: film french france against bardot paris poster
Topic 10: germany german war nazi letter christian book
Topic 11: east peace prize award timor quebec belo
Topic 12: n't life show told very love television
Topic 13: years year time last church world people
Topic 14: mother teresa heart calcutta charity nun hospital
Topic 15: city salonika capital buddhist cultural vietnam byzantine
Topic 16: music tour opera singer israel people film
Topic 17: church catholic be

2、利用LDA分析影评语料的话题组成，输出前10个话题。

In [12]:
import nltk
from nltk.corpus import movie_reviews
import random
import string
import numpy as np

documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
all_words=all_words.most_common(2000) #词频表按频率排序
badwords=nltk.corpus.stopwords.words('english')	#去停止词
word_features  =[w for (w,f) in all_words if w not in badwords]
features = np.zeros([len(documents),len(word_features)],dtype=float)
for n in range(len(documents)):
    document_words = set(documents[n][0])
    for  m in range(len(word_features)):
        if word_features[m] in document_words:
            features[n,m] = 1
target=[c for (d,c) in documents]

train_set=features[:1000,:]
target_train=target[:1000]
test_set=features[1000:,:]
target_test=target[1000:]

train_set

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [17]:
from gensim import corpora, models
import jieba.posseg as jp, jieba
import numpy as np
import lda
import nltk
from nltk.corpus import movie_reviews
import random
import string
#需要将datasets更换为自定义的数据集

documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
all_words=all_words.most_common(2000) #词频表按频率排序
badwords=nltk.corpus.stopwords.words('english')	#去停止词
word_features  =[w for (w,f) in all_words if w not in badwords]
texts=[]
for word in word_features:
    texts.append(word)
    
# 构造词典
dictionary = corpora.Dictionary(texts)
# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】
corpus = [dictionary.doc2bow(words) for words in texts]

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
# 打印所有主题，每个主题显示5个词
for topic in lda.print_topics(num_words=5):
    print(topic)
# 主题推断
print(lda.inference(corpus))

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

3、利用LDA分析Sogou语料的话题组成，输出前10个话题。

In [15]:
from gensim import corpora, models
import jieba.posseg as jp, jieba
import numpy as np
import lda
#需要将datasets更换为自定义的数据集
#X = lda.datasets.load_reuters()
#vocab = lda.datasets.load_reuters_vocab()
#titles = lda.datasets.load_reuters_titles()
#X.shape
#X.sum()
# 文本集


#构建停用词表
#停用词表来源：https://github.com/goto456/stopwords
stop_words = r'C:/Users/28347/Desktop/NLP/stopwords-master/hit_stopwords.txt'
stopwords = codecs.open(stop_words,'r',encoding='utf8').readlines()
stopwords = [ w.strip() for w in stopwords ]

#结巴分词后的停用词性 [标点符号、连词、助词、副词、介词、时语素、‘的’、数词、方位词、代词]

stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']

#对一篇文章分词、去停用词

def tokenization(filename):
    result = []
    with open(filename, 'r') as f:
        text = f.read()
        words = pseg.cut(text)
    for word, flag in words:
        if flag not in stop_flag and word not in stopwords:
            result.append(word)
    return result

filenames = ['C:/Users/28347/Desktop/NLP/Sogou/C000008/10.txt', 
             'C:/Users/28347/Desktop/NLP/Sogou/C000008/11.txt',
             'C:/Users/28347/Desktop/NLP/Sogou/C000008/12.txt',
             'C:/Users/28347/Desktop/NLP/Sogou/C000008/13.txt',
             'C:/Users/28347/Desktop/NLP/Sogou/C000008/14.txt'
            ]
texts = []
for each in filenames:
    texts.append(tokenization(each))
    
# 构造词典
dictionary = corpora.Dictionary(texts)
# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】
corpus = [dictionary.doc2bow(words) for words in texts]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
# 打印所有主题，每个主题显示5个词
for topic in lda.print_topics(num_words=5):
    print(topic)
# 主题推断
print(lda.inference(corpus))

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(721 unique tokens: ['CEO', 'DCM', 'Paypal', 'VISA', 'VISA卡']...) from 5 documents (total 1393 corpus positions)
INFO:gensim.models.ldamodel:using symmetric alpha at 0.5
INFO:gensim.models.ldamodel:using symmetric eta at 0.5
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 5 documents, updating model once every 5 documents, evaluating perplexity every 5 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:-7.256 per-word bound, 152.9 perplexity estimate based on a held-out corpus of 5 documents with 1393 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #5/5
INFO:gensim.models.ldamodel:topic #0 (0.500): 0.027*"支付" + 0.021*"公司" + 0.014*"快" + 0.014*"钱" + 0.011*"用户" + 0.00

(0, '0.027*"支付" + 0.021*"公司" + 0.014*"快" + 0.014*"钱" + 0.011*"用户"')
(1, '0.013*"支付" + 0.013*"快" + 0.012*"钱" + 0.011*"公司" + 0.010*"消费者"')
(array([[621.04047   ,   1.8926458 ],
       [ 79.692604  ,  13.290045  ],
       [  1.0402694 , 263.93134   ],
       [204.08127   ,   0.89753544],
       [211.34174   ,   0.63713783]], dtype=float32), None)
